In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import Proxy

# = = = = = = = = = = = = = = =

from openpyxl import load_workbook

wb = load_workbook('./part.xlsx')
ws = wb.active

for column in range(1, ws.max_column+1):
    if ws.cell(1, column).value == 'Part_Number':
        part_number_index = column
    elif ws.cell(1, column).value == 'Pic':
        pic_index = column
        
print('Part_Number索引：' + str(part_number_index))
print('Pic索引：' + str(pic_index))
print()
print('总数量：' + str(ws.max_row-1))
print()     

# = = = = = = = = = = = = = = =

import requests
from io import BytesIO
import PIL
from openpyxl.drawing import image

# = = = = = = = = = = = = = = =

from openpyxl.utils import get_column_letter

# = = = = = = = = = = = = = = =

import time

# = = = = = = = = = = = = = = =

work = Queue()
for row in range(2, ws.max_row+1):
    work.put_nowait(row)

def crawler():
    global ws
    
    while not work.empty():
        row = work.get_nowait()
        
        # = = = = = = = = = = = = = = = = = =
        
        part_number = ws.cell(row, part_number_index).value
        
        # = = = = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                pic = image.Image('./pic/' + part_number + '.png')
                
                # = = = = = = = = = = = = = = = = = =
                
                if pic.width >= pic.height:
                    pic.height = 50.0 / pic.width * pic.height
                    pic.width = 50.0
                else:
                    pic.width = 50.0 / pic.height * pic.width
                    pic.height = 50.0
                    
                # = = = = = = = = = = = = = = = = = =
                
                ws.add_image(pic, get_column_letter(pic_index) + str(row))
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(3)
                continue
                
        # = = = = = = = = = = = = = = = = = =
        
        if part_number != None and part_number != '':
            print(part_number +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))
        
# = = = = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
wb.save('./part-pic.xlsx')
print()
print('爬虫结束')

Part_Number索引：3
Pic索引：8

总数量：364

3310003  <->  [ok] - 剩余数量：363
3310004  <->  [ok] - 剩余数量：362
3310006  <->  [ok] - 剩余数量：361
3310007  <->  [ok] - 剩余数量：360
3310008  <->  [ok] - 剩余数量：359
3310009  <->  [ok] - 剩余数量：358
3310010  <->  [ok] - 剩余数量：357
3310011  <->  [ok] - 剩余数量：356
3310012  <->  [ok] - 剩余数量：355
3310013  <->  [ok] - 剩余数量：354
3310014  <->  [ok] - 剩余数量：353
3310015  <->  [ok] - 剩余数量：352
3310016  <->  [ok] - 剩余数量：351
3310019  <->  [ok] - 剩余数量：350
3310022  <->  [ok] - 剩余数量：349
3310023  <->  [ok] - 剩余数量：348
3310024  <->  [ok] - 剩余数量：347
3310025  <->  [ok] - 剩余数量：346
3310026  <->  [ok] - 剩余数量：345
3310028  <->  [ok] - 剩余数量：344
3310029  <->  [ok] - 剩余数量：343
3310030  <->  [ok] - 剩余数量：342
3310031  <->  [ok] - 剩余数量：341
3310032  <->  [ok] - 剩余数量：340
3310033  <->  [ok] - 剩余数量：339
3310035  <->  [ok] - 剩余数量：338
3310037  <->  [ok] - 剩余数量：337
3310038  <->  [ok] - 剩余数量：336
3310039  <->  [ok] - 剩余数量：335
3310040  <->  [ok] - 剩余数量：334
3310041  <->  [ok] - 剩余数量：333
3310042  <->  [ok] - 剩余数量：332
331004


爬虫结束
